In [10]:
%%capture
%matplotlib qt5
%matplotlib qt5


plot_sensors = 'grad' # 'mag', 'grad' or True (all sensors)
window = 10 #in seconds
start = 0.0 #start point in seconds

block1 = scipy.io.loadmat('/Users/valery/MEG/Cases/B1C2/ASPIRE/results/\
sources_B1C2_ii_run1_raw_tsss_mc_art_corr_data_\
block001_ICA_based_grad.mat')

block2 = scipy.io.loadmat('/Users/valery/MEG/Cases/B1C2/ASPIRE/results/\
sources_B1C2_ii_run1_raw_tsss_mc_art_corr_data_\
block002_ICA_based_grad.mat')
block2['spikeind'][0,:] += 600_000
spikes = block1['spikeind'][0,:].tolist() + block2['spikeind'][0,:].tolist()


data = mne.io.read_raw_fif( "/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr.fif", preload=False, verbose=False)
#data.pick_types(meg=plot_sensors, eeg=False,stim=False,eog=False,ecg=False,emg=False,misc=False)

onset = [(data.first_samp+i)/1000 for i in spikes]
data.annotations.append(onset, np.repeat(0.00001, len(onset)), [' ']*len(onset))

data.plot(duration=window, group_by='position', start=start, lowpass=100, highpass=9)


In [1]:
def create_epochs(raw_filt, detections, tmin=-1, tmax=1, picks_raw=True):
    """ Here we create epochs for events"""
    import mne
    import numpy as np
    eve_id = 1
    eve_name = 'ICA_det'
    

    raw_filt.load_data()
    
    new_events, eve = [], []
    first_samp = raw_filt.first_samp

    for spike_time in detections:
        eve = [int(round(spike_time + first_samp)), 0, eve_id]
        new_events.append(eve)
    
    ch_name = 'ICA'
    if ch_name not in raw_filt.info['ch_names']:
        stim_data = np.zeros((1, len(raw_filt.times)))
        info_sp = mne.create_info([ch_name], raw_filt.info['sfreq'], ['stim'])
        stim_sp = mne.io.RawArray(stim_data, info_sp, verbose=False)
        raw_filt.add_channels([stim_sp], force_update_info=True)

    raw_filt.add_events(new_events, stim_channel=ch_name, replace=True)
    events= mne.find_events(raw_filt, stim_channel=ch_name, verbose=False)
    event_id = {eve_name: eve_id}
    picks = mne.pick_types(raw_filt.info, meg=picks_raw, eeg=False, eog=False)
    epochs = mne.Epochs(raw_filt, events, event_id,  tmin, tmax, baseline=None, picks=picks, preload=True, verbose=False)
    return epochs
    del raw_filt, picks, event_id

In [3]:
import scipy.io
import mne
import numpy as np
import pandas as pd
block1 = scipy.io.loadmat('/Users/valery/MEG/Cases/B1C2/ASPIRE/results/\
sources_B1C2_ii_run1_raw_tsss_mc_art_corr_data_\
block001_ICA_based_grad.mat')

block2 = scipy.io.loadmat('/Users/valery/MEG/Cases/B1C2/ASPIRE/results/\
sources_B1C2_ii_run1_raw_tsss_mc_art_corr_data_\
block002_ICA_based_grad.mat')
block2['spikeind'][0,:] += 600_000
spikes = block1['spikeind'][0,:].tolist() + block2['spikeind'][0,:].tolist()



In [5]:
tsss_file = "/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr.fif"
data = mne.io.read_raw_fif(tsss_file, preload=False, verbose=False)
epochs = create_epochs(data, spikes, tmin=-0.5, tmax=0.5, picks_raw=True)
epochs.save("/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr_epochs.fif", overwrite=True)

raw_data = epochs.pick_types(meg=True, eeg=False,stim=False, eog=False).get_data()
raw_data_reshaped = raw_data.transpose(1,0,2).reshape(raw_data.shape[1],-1)

npy_file = "/Users/valery/MEG/Cases/B1C2/Spyking_circus/B1C2_B1C2_ii_run1_raw_tsss_mc_art_corr/only_epochs/B1C2_ii_run1_raw_tsss_mc_art_corr.fif"
np.save("%s_0.npy"%(npy_file[:-4]), (raw_data_reshaped).astype(float))

<ipython-input-5-6b2eabb860f9>:2: RuntimeWarning: This filename (/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  data = mne.io.read_raw_fif(tsss_file, preload=False, verbose=False)


Overwriting existing file.


<ipython-input-5-6b2eabb860f9>:4: RuntimeWarning: This filename (/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save("/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr_epochs.fif", overwrite=True)


In [ ]:
%%capture
%matplotlib qt5
%matplotlib qt5

epochs.plot(n_epochs=2)

In [ ]:
#tsss_file = "/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr.fif"
#data = mne.io.read_raw_fif(tsss_file, preload=True, verbose=False)

#raw_data = data.pick_types(meg=True, eeg=False,stim=False, eog=False).get_data()
#tsss_file = "/Users/valery/MEG/Cases/B1C2/Spyking_circus/B1C2_B1C2_ii_run1_raw_tsss_mc_art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr.fif"
#np.save("%s_1.npy"%(tsss_file[:-4]), (raw_data).astype(float))

#raw_data = epochs.pick_types(meg=True, eeg=False,stim=False, eog=False).get_data()
#tsss_file = "/Users/valery/MEG/Cases/B1C2/Spyking_circus/B1C2_B1C2_ii_run1_raw_tsss_mc_art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr.fif"
#np.save("%s_0.npy"%(tsss_file[:-4]), (raw_data).astype(float))

In [2]:
#tsss_file = "/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr_epochs.fif"
import mne
epochs = mne.read_epochs("/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr_epochs.fif", preload=False)

Reading /Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr_epochs.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
778 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


<ipython-input-2-f7540aaee048>:3: RuntimeWarning: This filename (/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs("/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr_epochs.fif", preload=False)


In [1]:
import run_circus_ASPIRE as run_circus
import scipy.io
import circus_templates_ASPIRE as circus_templates
import numpy as np
import pandas as pd
import mne
import traceback
import shutil
dir_case = "/Users/valery/MEG/Cases/B1C2/"
dir_SPC = dir_case + "Spyking_circus/B1C2_B1C2_ii_run1_raw_tsss_mc_art_corr/"
SPC_fname = "B1C2_ii_run1_raw_tsss_mc_art_corr.fif"
nmpy_file = 'B1C2_ii_run1_raw_tsss_mc_art_corr_0.npy'
epochs_fname = dir_case +  "art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr_epochs.fif"
for n_t in [450]:
    for mad in [6.0]:
        try:
            #shutil.copy(dir_SPC +'full_case/' + nmpy_file, dir_SPC)
            #sc = run_circus.Circus(dir_case, "B1C2", dir_SPC, SPC_fname, N_t=n_t, cc_merge=0.97, MAD=mad, cut_off=9)
            #sc.params_iterations(run_spc=True, only_fitting = False, sensors=['grad'])
            
            #shutil.copy(dir_SPC +'only_epochs/' + nmpy_file, dir_SPC)
            sc = run_circus.Circus(dir_case, "B1C2", dir_SPC, SPC_fname, N_t=n_t, cc_merge=0.97, MAD=mad, cut_off=9)
            sc.params_iterations(run_spc=False, only_fitting = True, sensors=['grad'])

            for sens in ['grad']:
                temp = circus_templates.Templates(dir_case, "B1C2", SPC_fname, sc.sensors_params[sens], sensors=sens, n_sp=3, N_t=n_t, cc_merge=0.97, MAD=mad)
                temp.plot_all_templates(epochs_fname, 'Mac')
        except Exception: traceback.print_exc()

Folder: /Users/valery/MEG/Cases/B1C2/Spyking_circus/B1C2_B1C2_ii_run1_raw_tsss_mc_art_corr/
File: B1C2_ii_run1_raw_tsss_mc_art_corr_0.npy
Parameters: N_t = 450, Cut off = 9, Threshold = 6.0, cc_merge = 0.97



       Sensors  [█████████████████████████████████████████████] 1/1 [00:02<00:02, 2.09s/it]

     Templates  [█████████████████████████████████████████████] 8/8 [03:14<00:21, 24.20s/it]

In [11]:
%%capture
evocked_0 = epochs[6].load_data().pick_types(meg='grad').filter(1., 45., fir_design='firwin').average()

In [4]:
%%capture
%matplotlib qt5
%matplotlib qt5
ch, t = evocked_0.get_peak(tmin=-0.2,tmax=0.2)
evocked_0.plot_topo(vline=[t,0])

The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


Traceback (most recent call last):
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 216, in process
    func(*args, **kwargs)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/mne/viz/topo.py", line 278, in _plot_topo_onpick
    show_func(ax, ch_idx)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/mne/viz/topo.py", line 455, in _plot_timeseries
    linestyle='--')
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/pyplot.py", line 2418, in axvline
    return gca().axvline(x=x, ymin=ymin, ymax=ymax, **kwargs)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/axes/_axes.py", line 931, in axvline
    scalex = (xx < xmin) or (xx > xmax)
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


Traceback (most recent call last):
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 216, in process
    func(*args, **kwargs)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/mne/viz/topo.py", line 278, in _plot_topo_onpick
    show_func(ax, ch_idx)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/mne/viz/topo.py", line 455, in _plot_timeseries
    linestyle='--')
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/pyplot.py", line 2418, in axvline
    return gca().axvline(x=x, ymin=ymin, ymax=ymax, **kwargs)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/axes/_axes.py", line 931, in axvline
    scalex = (xx < xmin) or (xx > xmax)
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


Traceback (most recent call last):
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 216, in process
    func(*args, **kwargs)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/mne/viz/topo.py", line 278, in _plot_topo_onpick
    show_func(ax, ch_idx)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/mne/viz/topo.py", line 455, in _plot_timeseries
    linestyle='--')
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/pyplot.py", line 2418, in axvline
    return gca().axvline(x=x, ymin=ymin, ymax=ymax, **kwargs)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/axes/_axes.py", line 931, in axvline
    scalex = (xx < xmin) or (xx > xmax)
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


Traceback (most recent call last):
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 216, in process
    func(*args, **kwargs)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/mne/viz/topo.py", line 278, in _plot_topo_onpick
    show_func(ax, ch_idx)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/mne/viz/topo.py", line 455, in _plot_timeseries
    linestyle='--')
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/pyplot.py", line 2418, in axvline
    return gca().axvline(x=x, ymin=ymin, ymax=ymax, **kwargs)
  File "/anaconda3/envs/mne/lib/python3.6/site-packages/matplotlib/axes/_axes.py", line 931, in axvline
    scalex = (xx < xmin) or (xx > xmax)
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


In [12]:
evocked_0.get_peak(tmin=-0.2,tmax=0.2)[1]

-0.075

In [14]:
best = temp.templates[abs(temp.templates.Amplitudes-1)<0.01]
best.reset_index(drop=True, inplace=True)

In [20]:
times = best[best.Template == 'temp_0'].Time.tolist()

In [10]:
epochs[0].load_data().crop(-0.4,0.0)

Loading data for 1 events and 1001 original time points ...


<EpochsFIF  |   1 events (all good), -0.4 - 0 sec, baseline off, ~6.5 MB, data loaded,
 'ICA_det': 1>

In [16]:
epochs[0].events

array([[40160,     0,     1]])

In [29]:
times

[124061, 283648, 534692, 654508, 714074]

In [7]:
%%capture
%matplotlib qt5
%matplotlib qt5


plot_sensors = 'grad' # 'mag', 'grad' or True (all sensors)
window = 10 #in seconds
start = 0.0 #start point in seconds

block1 = scipy.io.loadmat('/Users/valery/MEG/Cases/B1C2/ASPIRE/results/\
sources_B1C2_ii_run1_raw_tsss_mc_art_corr_data_\
block001_ICA_based_grad.mat')

block2 = scipy.io.loadmat('/Users/valery/MEG/Cases/B1C2/ASPIRE/results/\
sources_B1C2_ii_run1_raw_tsss_mc_art_corr_data_\
block002_ICA_based_grad.mat')
block2['spikeind'][0,:] += 600_000
spikes = block1['spikeind'][0,:].tolist() + block2['spikeind'][0,:].tolist()

#tsss_file = "/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr_epochs.fif"
#data = mne.io.read_raw_fif(tsss_file, preload=True, verbose=False)
#data.pick_types(meg=plot_sensors, eeg=False,stim=False,eog=False,ecg=False,emg=False,misc=False)

data = mne.io.read_raw_fif( "/Users/valery/MEG/Cases/B1C2/art_corr/B1C2_ii_run1_raw_tsss_mc_art_corr.fif", preload=True, verbose=False)
data.pick_types(meg=plot_sensors, eeg=False,stim=False,eog=False,ecg=False,emg=False,misc=False)

onset = temp.templates['Spiketimes']/1000 + data.first_samp/1000
data.annotations.append(onset, np.repeat(0.00001, len(onset)), temp.templates['Template'].tolist())

onset = [(data.first_samp+i)/1000 for i in spikes]
data.annotations.append(onset, np.repeat(0.00001, len(onset)), [' ']*len(onset))

#onset = [data.first_samp/1000 + i*0.4 +0.2  for i in range(len(spikes))]
#data.annotations.append(onset, np.repeat(0.00001, len(onset)), [' ']*len(onset))

data.plot(duration=window, group_by='position', start=start, lowpass=45, highpass=1)

In [5]:
ep = np.load(dir_SPC +'only_epochs/' + nmpy_file)

In [14]:
eppp = epochs[1].get_data()[0,:,:]

Loading data for 1 events and 1001 original time points ...


In [17]:
print([i for i in range(306) if (ep[i,1002]-eppp[i,1]) != 0])

[]


In [5]:
epp = epochs[0:10].get_data()[0:10,:,:]

Loading data for 10 events and 1001 original time points ...


In [17]:
epochs.get_data().shape

Loading data for 778 events and 2001 original time points ...


(778, 306, 2001)

In [8]:
eppp = epp.reshape(306,10010)

Loading data for 1 events and 1001 original time points ...


5.235066159908211e-12

In [13]:
a = np.array([[[1,2,3], 
               [4,5,6]],
              [[7,8,9], 
               [10,11,12]]])

(a[0,0,:],a[0,1,:])

(array([1, 2, 3]), array([4, 5, 6]))

In [27]:
a.transpose(1,0,2).reshape(2,-1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

In [24]:
a.reshape?

Docstring:
a.reshape(shape, order='C')

Returns an array containing the same data with a new shape.

Refer to `numpy.reshape` for full documentation.

See Also
--------
numpy.reshape : equivalent function

Notes
-----
Unlike the free function `numpy.reshape`, this method on `ndarray` allows
the elements of the shape parameter to be passed in as separate arguments.
For example, ``a.reshape(10, 11)`` is equivalent to
``a.reshape((10, 11))``.
Type:      builtin_function_or_method
